# Simulation

> This module has all the functions required for the inventory simulation functionality

In [ ]:
#| default_exp sdsim

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): return 10

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()